In [67]:
from google.cloud import bigquery
import re
import os
import pandas as pd
from google.cloud import storage
import io
project_id = "interseguro-data"
client = bigquery.Client(project_id)

In [203]:
query = """
SELECT
TABLE_NAME
,COLUMN_NAME
,COLUMN_ID
,1 AS INDICADOR
FROM `interseguro-data.acsele_data.ALL_TAB_COLUMNS_raw`
WHERE TABLE_NAME IN ('RECIBIRDOCUMENTO', 'DERIVARCOMERCIAL')
ORDER BY TABLE_NAME,COLUMN_ID ASC
"""

In [218]:
def sqlbigquery(filename):
    with open(filename,"r") as file:
        statement = file.readlines()
        statement = " ".join(statement)
    print(f" {filename}")
    return statement

def read_table_bq(sql):
    df = pd.read_gbq(sql, project_id=project_id, dialect='standard')
    return df

def load_data(df,filename):
    par_project_output = 'interseguro-data'
    par_bucket_output = 'interseguro-datalake-prod'
    client_output = storage.Client(par_project_output)
    bucket_output = client_output.bucket(par_bucket_output)
    bytes_writer = io.BytesIO()
    df_bytes = df.to_parquet(bytes_writer,engine="pyarrow")
    data_length = bytes_writer.tell()
    print(f"[INFO] size file : {data_length}")
    bytes_writer.seek(0)
    blob_n = bucket_output.blob(f"SPARK_TABLE/ACSELE/{filename}/{filename}.parquet")
    blob_n.upload_from_string(
        data=bytes_writer.getvalue(),
        content_type='application/octet-stream',
        client=client_output
    )


In [79]:
df = read_table_bq (query)

C:\Users\jberriot\AppData\Local\Temp\ipykernel_8648\52575413.py:9: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(sql, project_id=project_id, dialect='standard')


In [156]:
pivot_df = df.pivot(index='COLUMN_NAME', columns='TABLE_NAME', values='INDICADOR')
pivot_df = pivot_df.fillna(0)
pivot_df.to_excel('testeo.xlsx')
columnas = pivot_df.columns
print(columnas)
df_temp = pd.read_excel('testeo.xlsx')

Index(['DERIVARCOMERCIAL', 'RECIBIRDOCUMENTO'], dtype='object', name='TABLE_NAME')


In [216]:
def aplicar_condicion(row):
    resultado = {}
    for col in columnas:
        if row.name == num_filas -1 :
            resultado[col] = ',' + row['COLUMN_NAME'] + '\nFROM INTERSEGURO.'+col + '\nUNION ALL ' 
        elif row[col] == 1:
            #resultado[col] = 'SELECT \n'+ row['COLUMN_NAME'] if row.name == 0 else ',' + row['COLUMN_NAME']
            resultado[col] = f"SELECT \n'{col}' AS ESTADO \n,"+ row['COLUMN_NAME'] if row.name == 0 else ',' + row['COLUMN_NAME']
        else:
            resultado[col] = 'SELECT NULL AS ' + row['COLUMN_NAME'] if row.name == 0 else ',NULL AS ' + row['COLUMN_NAME']
    return pd.Series(resultado)

In [219]:
def generate_file(df):
    with open('output.sql', 'w') as file:
        for col in columnas:
            for value in df[col]:
                file.write(str(value) + '\n')
                
num_filas = df_temp.shape[0]
nuevas_columnas = df_temp.apply(aplicar_condicion, axis=1)
df_t = pd.concat([df_temp['COLUMN_NAME'], nuevas_columnas], axis=1)                
generate_file(df_t)            